### Data Ingestion [Load data -> Apply chunking -> Convert into embeddings -> Store it into Vector_DB]

In [ ]:
### document Structure

from langchain_core.documents import Document # import Document class

In [ ]:
# define object of Document class

doc = Document(
    page_content = "This is the content of the document.",
    metadata = {                                              # in order to make the retrieval more efficient, we can add metadata (also it helps in filtering the documents based on certain criteria)
        "source": "rag_document.txt",
        "pages" : 10,
        "author": "Navneeth"
    }  
)